Setup the openai api key in env

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

To use OpenAI language models

In [5]:
import dspy
gpt3_turbo = dspy.OpenAI(model='gpt-3.5-turbo-1106', max_tokens=300)
dspy.configure(lm=gpt3_turbo)

#### Inline DSPy Signatures
- Question Answering: "question -> answer"

- Sentiment Classification: "sentence -> sentiment"

- Summarization: "document -> summary"

#### Your signatures can also have multiple input/output fields.

- Retrieval-Augmented Question Answering: "context, question -> answer"

- Multiple-Choice Question Answering with Reasoning: "question, choices -> reasoning, selection"

Example A: Sentiment Classification

In [6]:
sentence = "it's a charming and often affecting journey."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment')
classify(sentence=sentence).sentiment

'Positive'

Example B: Summarization

In [12]:
# Example from the XSum dataset.
document = """The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."""

summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)

print(response.summary)

The 21-year-old player made seven appearances and scored one goal for West Ham in the Europa League. He had loan spells in League One with Blackpool and Colchester United, scoring twice for the latter. He has now signed a contract with Barnsley, although the length of the contract has not been disclosed.


Many DSPy modules (except dspy.Predict) return auxiliary information by expanding your signature under the hood.

For example, `dspy.ChainOfThought` also adds a rationale field that includes the LM's reasoning before it generates the output summary.

In [8]:
print("Rationale:", response.rationale)

Rationale: produce the summary. We need to highlight the player's performance for West Ham, his loan spells in League One, and his new contract with Barnsley.


#### Class-based DSPy Signatures

For some advanced tasks, you need more verbose signatures. This is typically to:

- Clarify something about the nature of the task (expressed below as a `docstring`).

- Supply hints on the nature of an input field, expressed as a `desc` keyword argument for `dspy.InputField`.

- Supply constraints on an output field, expressed as a `desc` keyword argument for `dspy.OutputField`.

Example C: Classification

In [9]:
class Emotion(dspy.Signature):
    """Classify emotion among sadness, joy, love, anger, fear, surprise."""
    
    sentence = dspy.InputField()
    sentiment = dspy.OutputField()

sentence = "i started feeling a little vulnerable when the giant spotlight started blinding me"  # from dair-ai/emotion

classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='Fear'
)

Example D: A metric that evaluates faithfulness to citations

In [13]:
class CheckCitationFaithfulness(dspy.Signature):
    """Verify that the text is based on the provided context."""

    context = dspy.InputField(desc="facts here are assumed to be true")
    text = dspy.InputField()
    faithfulness = dspy.OutputField(desc="True/False indicating if text is faithful to context")

context = "The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."

text = "Lee scored 3 goals for Colchester United."

faithfulness = dspy.ChainOfThought(CheckCitationFaithfulness)
faithfulness(context=context, text=text)

Prediction(
    rationale="produce the faithfulness. We know that Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. However, the text states that Lee scored 3 goals for Colchester United, which is not consistent with the context.",
    faithfulness='False'
)